In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('BBDD_actualizado.csv')

In [3]:
def filtrar_jugadores(df, edad_max=None, posicion=None, proyeccion=None, pierna=None, goles=None, asistencias=None, porterias=None):
    condiciones = []
    if edad_max is not None:
        condiciones.append(df['Edad'] < edad_max)
    if posicion is not None:
        condiciones.append(df['Posición'] == posicion)
    if proyeccion is not None:
        condiciones.append(df['Proyección'] == proyeccion)
    if pierna is not None:
        condiciones.append(df['Pierna Buena'] == pierna)
    if goles is not None:
        condiciones.append(df['Goles'] >= goles)
    if asistencias is not None:
        condiciones.append(df['Asistencias'] >= asistencias)
    if porterias is not None:
        condiciones.append(df['Porterías a Cero'] >= porterias)

    if condiciones:
        jugadores_filtrados = df.loc[pd.concat(condiciones, axis=1).all(axis=1)]
    else:
        jugadores_filtrados = df

    return jugadores_filtrados

# Ejemplo de uso
jugadores_filtrados = filtrar_jugadores(df, edad_max=20, posicion='Delantero', pierna = 'Izquierda', goles = 15, asistencias = 10)


In [3]:
!pip install transformers
!pip install streamlit
!pip install gradio

In [4]:
import gradio as gr
import re
import unicodedata
import pandas as pd
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM

def filtrar_jugadores(df, edad_max=None, posicion=None, proyeccion=None, pierna=None, goles=None, asistencias=None, porterias=None):
    condiciones = []
    if edad_max is not None:
        condiciones.append(df['Edad'] < edad_max)
    if posicion is not None:
        condiciones.append(df['Posición'] == posicion)
    if proyeccion is not None:
        condiciones.append(df['Proyección'] == proyeccion)
    if pierna is not None:
        condiciones.append(df['Pierna Buena'] == pierna)
    if goles is not None:
        condiciones.append(df['Goles'] >= goles)
    if asistencias is not None:
        condiciones.append(df['Asistencias'] >= asistencias)
    if porterias is not None:
        condiciones.append(df['Porterías a Cero'] >= porterias)

    if condiciones:
        jugadores_filtrados = df.loc[pd.concat(condiciones, axis=1).all(axis=1)]
    else:
        jugadores_filtrados = df

    return jugadores_filtrados

def normalize_text(text):
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

class FootballBot:
    def __init__(self, df):
        self.df = df

    def get_players(self, edad_max=None, posicion=None, proyeccion=None, pierna=None, goles=None, asistencias=None, porterias=None):
        return filtrar_jugadores(self.df, edad_max, posicion, proyeccion, pierna, goles, asistencias, porterias)

    def get_scout_comments(self, players):
        comments = {}
        for index, row in players.iterrows():
            comments[row['Nombre']] = row['Comentario del ojeador']
        return comments

    def generate_response(self, query):
        inputs = tokenizer(query, return_tensors="pt")
        outputs = model.generate(inputs['input_ids'], max_length=150)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

    def handle_query(self, query):
        query = normalize_text(query)

        edad_max = None
        posicion = None
        proyeccion = None
        pierna = None
        goles = None
        asistencias = None
        porterias = None

        # Buscar y extraer parámetros de la consulta
        edad_match = re.search(r'menor de (\d+) anos', query)
        if edad_match:
            edad_max = int(edad_match.group(1))

        if 'delantero' in query:
            posicion = 'Delantero'
        elif 'centrocampista' in query:
            posicion = 'Centrocampista'
        elif 'defensa' in query:
            posicion = 'Defensa'
        elif 'portero' in query:
            posicion = 'Portero'

        proyeccion_match = re.search(r'proyeccion (elite|alta|media|baja)', query)
        if proyeccion_match:
            proyeccion = proyeccion_match.group(1)

        if 'zurdo' in query:
            pierna = 'Izquierda'
        elif 'diestro' in query:
            pierna = 'Derecha'

        goles_match = re.search(r'mas de (\d+) goles', query)
        if goles_match:
            goles = int(goles_match.group(1))

        asistencias_match = re.search(r'mas de (\d+) asistencias', query)
        if asistencias_match:
            asistencias = int(asistencias_match.group(1))

        porterias_match = re.search(r'mas de (\d+) porterias a cero', query)
        if porterias_match:
            porterias = int(porterias_match.group(1))

        players = self.get_players(edad_max, posicion, proyeccion, pierna, goles, asistencias, porterias)

        if 'comentarios del ojeador' in query:
            comments = self.get_scout_comments(players)
            return comments

        player_list = players[['Nombre', 'Edad', 'Posición', 'Equipo', 'Proyección', 'Pierna Buena', 'Goles', 'Asistencias', 'Porterías a Cero', 'Comentarios del Ojeador']].to_dict('records')
        return player_list

# Cargar la base de datos
df = pd.read_csv('BBDD_actualizado.csv')

# Crear una instancia del bot con el DataFrame cargado
bot = FootballBot(df)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
#GPU Enabled use below 👇🏽
#model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="cuda")

def query_bot(query):
    response = bot.handle_query(query)
    search_result = ''
    for player in response:
      search_result = search_result + f"Jugador: {player.get('Nombre', 'N/A')}, {player.get('Edad', 'N/A')}, {player.get('Equipo', 'N/A')}, {player.get('Goles', 'N/A')}, {player.get('Asistencias', 'N/A')}, {player.get('Comentarios del Ojeador', 'N/A')}\n"

    combined_information = f"Query: {query}\n{search_result}"
    # Move tensors to GPU and generate response
    input_ids = tokenizer(combined_information, return_tensors="pt")#.to('cuda')
    response = model.generate(**input_ids, max_new_tokens=500)
    # Traduce la respuesta del modelo al español
    full_response = tokenizer.decode(response[0])

    return full_response

import gradio as gr

# Interfaz de Gradio
iface = gr.Interface(
    fn=query_bot,
    inputs="text",
    outputs="text",
    title="FootballBot",
    description="Consulta información sobre jugadores de fútbol."
)

iface.launch(share=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://52a2a823287f5688d8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [14]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s 

In [45]:
fb = FootballBot(df)
a = fb.handle_query('Devuélveme un delantero menor de 20 años que sea zurdo con más de 15 goles y más de 10 asistencias')

search_result = ''
for player in a:
  search_result = search_result + f"Jugador - Nombre: {player.get('Nombre', 'N/A')}, Edad: {player.get('Edad', 'N/A')}, Equipo: {player.get('Equipo', 'N/A')}, Goles: {player.get('Goles', 'N/A')}, Asistencias: {player.get('Asistencias', 'N/A')}, Porterías a Cero: {player.get('Porterías a Cero', 'N/A')}, Comentarios: {player.get('Comentarios del Ojeador', 'N/A')}\n"

In [46]:
print(search_result)

Jugador - Nombre: Miguel Santos, Edad: 18, Equipo: Santos FC United, Goles: 15, Asistencias: 15, Porterías a Cero: nan, Comentarios: Delantero letal, gran capacidad goleadora y visión para asistir. Decisivo en el área.
Jugador - Nombre: Fernando Pereira, Edad: 17, Equipo: Palmeiras Atlético, Goles: 19, Asistencias: 12, Porterías a Cero: nan, Comentarios: Delantero letal, gran capacidad goleadora y visión para asistir. Decisivo en el área.
Jugador - Nombre: Jaime Fernandez, Edad: 19, Equipo: Cruzeiro Atlético, Goles: 15, Asistencias: 12, Porterías a Cero: nan, Comentarios: Delantero letal, gran capacidad goleadora y visión para asistir. Decisivo en el área.
Jugador - Nombre: Diego Hernández, Edad: 18, Equipo: Botafogo Atlético, Goles: 15, Asistencias: 10, Porterías a Cero: nan, Comentarios: Delantero letal, gran capacidad goleadora y visión para asistir. Decisivo en el área.
Jugador - Nombre: Gabriel Ramos, Edad: 19, Equipo: Internacional City, Goles: 18, Asistencias: 13, Porterías a Ce

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
#GPU Enabled use below 👇🏽
#model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [44]:
query = 'Devuélveme un delantero menor de 20 años que sea zurdo con más de 15 goles y más de 10 asistencias'

In [18]:
combined_information = f"Desarrolla uno de estos jugadores de manera fluida:\n{search_result}"
# Move tensors to GPU and generate response
input_ids = tokenizer(combined_information, return_tensors="pt")#.to('cuda')
response = model.generate(**input_ids, max_new_tokens=500)
# Traduce la respuesta del modelo al español
full_response = tokenizer.decode(response[0])

In [49]:
def generar_respuesta(query, solucion):
    prompt = f"Choose the best player from below. Always choose de youngest one. Develop the characteristics of the selected player.\nUsuario: {query}\n{solucion}\n"
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=550, num_return_sequences=1)
    respuesta = print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return respuesta

In [50]:
generar_respuesta(query, search_result)

Choose the best player from below. Always choose de youngest one. Develop the characteristics of the selected player.
Usuario: Devuélveme un delantero menor de 20 años que sea zurdo con más de 15 goles y más de 10 asistencias
Jugador - Nombre: Miguel Santos, Edad: 18, Equipo: Santos FC United, Goles: 15, Asistencias: 15, Porterías a Cero: nan, Comentarios: Delantero letal, gran capacidad goleadora y visión para asistir. Decisivo en el área.
Jugador - Nombre: Fernando Pereira, Edad: 17, Equipo: Palmeiras Atlético, Goles: 19, Asistencias: 12, Porterías a Cero: nan, Comentarios: Delantero letal, gran capacidad goleadora y visión para asistir. Decisivo en el área.
Jugador - Nombre: Jaime Fernandez, Edad: 19, Equipo: Cruzeiro Atlético, Goles: 15, Asistencias: 12, Porterías a Cero: nan, Comentarios: Delantero letal, gran capacidad goleadora y visión para asistir. Decisivo en el área.
Jugador - Nombre: Diego Hernández, Edad: 18, Equipo: Botafogo Atlético, Goles: 15, Asistencias: 10, Porterías